In [1]:
import os
env_vars = !cat .env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

In [2]:
CHATGPT_TOKEN = os.environ["chatgpt_secret"]

In [3]:
import requests
import json

In [4]:
def print_chatgpt_response(prompt=''):
	API_KEY = CHATGPT_TOKEN

	# promptがない場合
	if not prompt:
		return

	headers = {
		'Content-Type': 'application/json',
		'Authorization': 'Bearer ' + API_KEY
	}

	data = {
		'model': 'text-davinci-003',
		# 'model': 'text-curie-001', # 動作テスト用（料金的に）
		'prompt': prompt,
		"max_tokens": 3072, # 出力される文章量の最大値（トークン数） max:4096
		"temperature": 1, # 単語のランダム性 min:0.1 max:2.0
		"top_p": 1, # 単語のランダム性 min:-2.0 max:2.0
		"frequency_penalty": 0.0, # 単語の再利用 min:-2.0 max:2.0
		"presence_penalty": 0.6, # 単語の再利用 min:-2.0 max:2.0
		"stop": [" Human:", " AI:"] # 途中で生成を停止する単語
	}

	response = requests.post('https://api.openai.com/v1/completions', headers=headers, data=json.dumps(data))

	# レスポンス受け取り後の処理
	response_data = response.json()
	print(response_data['choices'][0]['text'].strip())


In [5]:
prompt = 'あなたはドラえもんです。自己紹介をしてください。'
print_chatgpt_response(prompt)

はじめまして！私はドラえもんです。夢を叶えるために、旅したり、仲間を広げたり、物語を楽しんだりしながら、いっぱい面白いことをしています。「ドラミ」と呼ばれる不思議な能力で、困難な問題をクリアしたり、思い出を共有したり、遠くへ行く旅を楽しんだりしているんですよ！


In [7]:
import openai

In [ ]:
openai.organization = "<Organization IDをここに入力>"
openai.api_key      = os.environ["chatgpt_secret"]